In [2]:
# ...existing code...
import requests
import json
from typing import Any

class MatchbookClient:
    def __init__(
        self,
        username: str,
        password: str,
        base_urls: dict | None = None,
        user_agent: str = "api-doc-test-client",
        default_service: str = "bpapi",
        timeout: float | None = 15.0,
    ):
        self.username = username
        self.password = password
        self.base_urls = base_urls or {
            "bpapi": "https://api.matchbook.com/bpapi/rest",
            "edge": "https://api.matchbook.com/edge/rest",
        }
        self.default_service = default_service
        self.timeout = timeout

        self.session = requests.Session()
        self.session.headers.update({
            "accept": "application/json",
            "User-Agent": user_agent,
            "content-type": "application/json;charset=UTF-8",
        })
        self.token = None

    def _build_url(self, service: str, path: str) -> str:
        base = self.base_urls[service].rstrip("/")
        return f"{base}/{path.lstrip('/')}"

    def login(self):
        r = self.session.post(
            self._build_url("bpapi", "security/session"),
            json={"username": self.username, "password": self.password},
            timeout=self.timeout,
        )
        r.raise_for_status()
        self.token = r.json()["session-token"]
        self.session.headers["session-token"] = self.token
        return self.token

    def logout(self):
        try:
            self.session.delete(
                self._build_url("bpapi", "security/session"),
                timeout=self.timeout,
            )
        finally:
            self.token = None
            self.session.headers.pop("session-token", None)

    def _needs_reauth(self, r):
        if r.status_code in (401, 403):
            return True
        try:
            body = r.json()
            if isinstance(body, dict):
                text = json.dumps(body)
                if "AUTHENTICATION_REQUIRED" in text or "INVALID_SESSION" in text:
                    return True
        except Exception:
            if "AUTHENTICATION_REQUIRED" in r.text or "INVALID_SESSION" in r.text:
                return True
        return False

    def request(
        self,
        method: str,
        path: str,
        *,
        service: str | None = None,
        retries: int = 1,
        ensure_login: bool = True,
        params: dict | None = None,
        json: Any = None,
        data: Any = None,
        headers: dict | None = None,
        files: dict | None = None,
        **kwargs,
    ):
        if ensure_login and not self.token:
            self.login()

        url = path if path.startswith("http") else self._build_url(service or self.default_service, path)
        if "timeout" not in kwargs and self.timeout is not None:
            kwargs["timeout"] = self.timeout

        r = self.session.request(
            method.upper(),
            url,
            params=params,
            json=json,
            data=data,
            headers=headers,  # merged with session headers by requests
            files=files,
            **kwargs,
        )
        if self._needs_reauth(r) and retries > 0:
            self.login()
            return self.request(
                method,
                path,
                service=service,
                retries=retries - 1,
                ensure_login=False,
                params=params,
                json=json,
                data=data,
                headers=headers,
                files=files,
                **kwargs,
            )
        return r

    def get(self, path: str, *, service: str | None = None, params: dict | None = None, **kwargs):
        return self.request("GET", path, service=service, params=params, **kwargs)

    def post(self, path: str, *, service: str | None = None, json: Any = None, data: Any = None, **kwargs):
        return self.request("POST", path, service=service, json=json, data=data, **kwargs)

    def delete(self, path: str, *, service: str | None = None, **kwargs):
        return self.request("DELETE", path, service=service, **kwargs)

# Usage
client = MatchbookClient(USERNAME, PASSWORD)

# # bpapi: session
# print(client.get("security/session", service="bpapi").json())

# # bpapi: balance
# print(client.get("account/balance", service="bpapi").json())

# edge: lookups with query params
print(client.get("lookups/sports", service="edge", params={"per-page": 50, "offset": 0}).json())

# # Absolute URL + no login
# print(client.get("https://api.matchbook.com/edge/rest/lookups/sports", ensure_login=False, params={"per-page": 10}).json())
# # ...existing code...

{'total': 48, 'per-page': 50, 'offset': 0, 'sports': [{'name': 'American Football', 'id': 1, 'type': 'SPORT', 'url-name': 'american-football'}, {'name': 'Athletics', 'id': 555636871580009, 'type': 'SPORT', 'url-name': 'athletics'}, {'name': 'Australian Rules', 'id': 112, 'type': 'SPORT', 'url-name': 'australian-rules'}, {'name': 'Auto Racing', 'id': 13, 'type': 'SPORT', 'url-name': 'auto-racing'}, {'name': 'Baseball', 'id': 3, 'type': 'SPORT', 'url-name': 'baseball'}, {'name': 'Basketball', 'id': 4, 'type': 'SPORT', 'url-name': 'basketball'}, {'name': 'Boxing', 'id': 14, 'type': 'SPORT', 'url-name': 'boxing'}, {'name': 'Chess', 'id': 1387652895550017, 'type': 'SPORT', 'url-name': 'chess'}, {'name': 'Cricket', 'id': 110, 'type': 'SPORT', 'url-name': 'cricket'}, {'name': 'Current Events', 'id': 11, 'type': 'SPORT', 'url-name': 'current-events'}, {'name': 'Cycling', 'id': 115, 'type': 'SPORT', 'url-name': 'cycling'}, {'name': 'Darts', 'id': 116, 'type': 'SPORT', 'url-name': 'darts'}, {'na

In [ ]:
# ...existing code...
import pandas as pd
from zoneinfo import ZoneInfo

HORSE_RACING_SPORT_ID = "24735152712200"

def fetch_horseracing_events_by_tag(client, tag_url_names: str = "uk", include_prices: bool = True) -> list[dict]:
    params = {
        "sport-ids": HORSE_RACING_SPORT_ID,
        "tag-url-names": tag_url_names,
        "states": "open,suspended",   # include suspended to see more markets
        "include-prices": include_prices,
        "odds-type": "DECIMAL",
        "price-depth": 3,
        "per-page": 200,
        "offset": 0,
    }
    events: list[dict] = []
    while True:
        r = client.get("events", service="edge", params=params)
        r.raise_for_status()
        data = r.json()
        page = data.get("events", []) or []
        events.extend(page)
        if not page or len(events) >= int(data.get("total", 0)):
            break
        params["offset"] += params["per-page"]
    return events


def create_market_data():

    events = fetch_horseracing_events_by_tag(client, tag_url_names="uk")

    rows = []
    for e in events:
        course = next((t.get("name") for t in e.get("meta-tags", []) or [] if t.get("type") == "LOCATION"), None)
        race_time = pd.to_datetime(e["start"], utc=True).tz_convert(ZoneInfo("Europe/London")).tz_localize(None)
        for m in e.get("markets", []) or []:
            mname = (m.get("name") or "").strip()
            is_win = mname.upper() == "WIN"
            is_place = mname.lower().startswith("place")
            if not (is_win or is_place):
                continue
            for r in m.get("runners", []) or []:
                prices = r.get("prices", []) or []
                backs = [p for p in prices if p.get("side") == "back"]
                lays  = [p for p in prices if p.get("side") == "lay"]
                best_back = max(backs, key=lambda p: p["decimal-odds"]) if backs else None
                best_lay  = min(lays,  key=lambda p: p["decimal-odds"]) if lays  else None
                rows.append({
                    "event_id": e["id"],
                    "course": course,
                    "race_time": race_time,
                    "market_id": m["id"],
                    "market_name": mname,  # "WIN" or "Place (n)"
                    "runner_id": r["id"],
                    "runner_name": r["name"],
                    "best_back_odds": best_back["decimal-odds"] if best_back else None,
                    "best_back_available": best_back["available-amount"] if best_back else None,
                    "best_lay_odds": best_lay["decimal-odds"] if best_lay else None,
                    "best_lay_available": best_lay["available-amount"] if best_lay else None,
                })

    df = pd.DataFrame(rows)
    df["runner_name"] = df["runner_name"].str.replace(r"^\s*\d+\s*\.?\s*", "", regex=True).str.strip()

    final_df = df[
        [
            "event_id","course","race_time","market_id","market_name",
            "runner_id","runner_name","best_back_odds","best_back_available","best_lay_odds","best_lay_available",
        ]
    ].sort_values(["race_time","course","event_id","market_name","runner_name"]).reset_index(drop=True)

    print("Courses found:", sorted(final_df["course"].dropna().unique()))
# display(final_df.head(30))
# ...existing code...

Courses found: ['Ayr', 'Chester', 'Newbury', 'Newmarket', 'Wolverhampton']


In [11]:
final_df['market_name'].value_counts()

market_name
WIN          440
Place (3)    437
Place (4)    411
Place (2)    357
Place (5)     50
Place (6)     50
Name: count, dtype: int64

In [13]:
final_df[final_df['race_time'] == '2025-09-20 13:00:00']

,event_id,course,race_time,market_id,market_name,runner_id,runner_name,best_back_odds,best_back_available,best_lay_odds,best_lay_available
0,31223512809500061,Newmarket,2025-09-20 13:00:00,31223512918000058,Place (2),31223512918502058,Annastarzy,5.50,4.01001,9.40,6.01000
1,31223512809500061,Newmarket,2025-09-20 13:00:00,31223512918000058,Place (2),31223512918704058,Aplaceinthesun,10.00,4.01000,26.00,4.01001
2,31223512809500061,Newmarket,2025-09-20 13:00:00,31223512918000058,Place (2),31223512918910058,Born To Bright,10.00,4.01000,1000.00,2.11000
3,31223512809500061,Newmarket,2025-09-20 13:00:00,31223512918000058,Place (2),31223512919113058,Forever True,2.42,7.01001,4.10,6.01001
4,31223512809500061,Newmarket,2025-09-20 13:00:00,31223512918000058,Place (2),31223512919310058,Loving Queen,2.98,4.01001,5.20,7.01001
5,31223512809500061,Newmarket,2025-09-20 13:00:00,31223512918000058,Place (2),31223512919511058,Nanoscience,3.60,7.01001,6.80,4.01001
6,31223512809500061,Newmarket,2025-09-20 13:00:00,31223512918000058,Place (2),31223512919708058,Romantic Symphony,1.77,2.10001,2.26,2.01000
7,31223512809500061,Newmarket,2025-09-20 13:00:00,31223512918000058,Place (2),31223512919905058,Tryst,3.45,2.01000,4.10,8.01001
8,31223512809500061,Newmarket,2025-09-20 13:00:00,31223512920115058,Place (3),31223512920606058,Annastarzy,3.30,2.01000,4.40,2.01000
9,31223512809500061,Newmarket,2025-09-20 13:00:00,31223512920115058,Place (3),31223512920903058,Aplaceinthesun,4.20,2.01000,8.80,2.01000
